In [1]:
pip install tokenizers

     |████████████████████████████████| 3.3 MB 28.7 MB/s 


In [2]:
pip install tqdm

In [3]:
pip install transformers

     |████████████████████████████████| 2.6 MB 33.4 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 636 kB 57.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 27.3 MB/s 


In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import plotly.express as pe
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer
from tqdm.notebook import tqdm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *

In [8]:
data_train = pd.read_csv('/content/data_train.csv')
data_test = pd.read_csv('/content/data_test.csv')

In [9]:
data_train.isnull().sum()

ID            0
author        0
Review       50
Sentiment     0
dtype: int64

In [10]:
data_test.isnull().sum()

ID         0
author     0
Review    24
dtype: int64

In [11]:
data_train.Review = data_train.Review.fillna('unavailable')
data_test.Review = data_test.Review.fillna('unavailable')

In [12]:
dataset_train = data_train[['Review','Sentiment']]
dataset_test = data_test['Review']

In [13]:
train, validation = train_test_split(dataset_train, test_size=0.2, random_state=1)

In [14]:
print(train.shape)
print(validation.shape)

(35280, 2)
(8820, 2)


In [15]:
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer, InputFeatures

In [16]:
model_bert = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=3)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [17]:
vocabulary = tokenizer.get_vocab()

In [18]:
tokenizer.save_pretrained('.')
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [19]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding()
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [20]:
def build_model(transformer, max_len=512):

    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, :]
    out = Dense(3, activation='softmax')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-8), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    
    return model

In [21]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [22]:
AUTO = tf.data.experimental.AUTOTUNE

EPOCHS = 2
BATCH_SIZE = 16  * strategy.num_replicas_in_sync
MAX_LEN = 30

In [23]:
x_train = fast_encode(train.Review.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(validation.Review.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(dataset_test.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = train.Sentiment.values
y_valid = validation.Sentiment.values

In [24]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(100)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [25]:
with strategy.scope():
    model = build_model(model_bert, max_len=MAX_LEN)
model.summary()

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 30)]              0         
_________________________________________________________________
tf_distil_bert_for_sequence_ TFSequenceClassifierOutpu 66955779  
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 3)                 0         
_________________________________________________________________
dense (Dense)                (None, 3) 

In [26]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

2205/2205 [==============================] - 6259s 3s/step - loss: 0.7596 - sparse_categorical_accuracy: 0.6507 - val_loss: 0.7054 - val_sparse_categorical_accuracy: 0.6822
Epoch 2/2
2205/2205 [==============================] - 6233s 3s/step - loss: 0.6212 - sparse_categorical_accuracy: 0.7355 - val_loss: 0.7605 - val_sparse_categorical_accuracy: 0.6855


In [27]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

Epoch 1/2
551/551 [==============================] - 1466s 3s/step - loss: 0.7164 - sparse_categorical_accuracy: 0.6781
Epoch 2/2
551/551 [==============================] - 1452s 3s/step - loss: 0.5332 - sparse_categorical_accuracy: 0.7796


In [28]:
prediction = model.predict(test_dataset, verbose=1)

1182/1182 [==============================] - 883s 746ms/step


In [29]:
prediction

array([[1.3043600e-01, 2.9014310e-02, 8.4054971e-01],
       [6.7610568e-01, 2.1174003e-01, 1.1215428e-01],
       [1.9630724e-01, 1.1440282e-04, 8.0357838e-01],
       ...,
       [9.3542814e-01, 5.0663158e-02, 1.3908778e-02],
       [3.0139205e-01, 4.0996328e-01, 2.8864461e-01],
       [1.3845517e-01, 4.7969133e-01, 3.8185346e-01]], dtype=float32)

In [30]:
pred = pd.DataFrame(prediction)

In [31]:
pred.to_csv('pred4.csv',index=False)